In [1]:
import numpy as np
import pandas as pd
from matplotlib.pylab import plt

In [2]:
train = pd.read_csv('./data/train.csv', delimiter=',', header=0)
test = pd.read_csv('./data/test.csv', delimiter=',', header=0)

In [ ]:
# for index, row in train.iterrows():
#    key = row['label']
#    print('Label: ',str(key))
#    data = np.array(row[1:]).reshape((28, 28))
#    plt.imshow(data, cmap=plt.get_cmap('gray'))
#    plt.show()

In [ ]:
# train['img'] = train.apply(lambda row: np.array(row[1:]).reshape((28, 28)), axis=1)

# Unfortunately it's not possible to store a 2d array in a Serie of pandas dataframe,
# I would require to use a Panel (i.e. Dataframe of Dataframe, I don't see how to do that with Scikit easily)
# So hopefully neural network model can take a row as input and interpret that as a 28x28 image array

Open questions:
- How to not trip the ML classifier when the digit is in a corner and not in the center ?
- What if colors are inverted ?

Ideas :
- Generate more data by translating data to the left/right/up/down
- Sharpen the image and add to the data with opencv/imagemagick
- scale up or down the data

In [3]:
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

from sklearn.model_selection import cross_val_score

from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

from keras.layers import Input, merge, Flatten, Activation, Reshape
from keras.layers.convolutional import MaxPooling2D, Convolution2D, AveragePooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization

from keras.callbacks import History, ModelCheckpoint, CSVLogger, EarlyStopping
from keras.optimizers import Adam

import time

/usr/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using Theano backend.
Using gpu device 0: GeForce GTX 1070 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5105)
/usr/lib/python3.6/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see 

In [ ]:
# Keras neural Network

# Get dimensions
nb_dim = train.shape[1] - 1 # 784 features
nb_classes = pd.get_dummies(train['label']).shape[1] #10 possible values for label
img_2Dshape = (1,28, 28)

# Callbacks
history = History()
checkpointer = ModelCheckpoint(filepath=time.strftime("%Y-%m-%d_%H%M-")+"DCNN_weights.hdf5",
                               verbose=1, save_best_only=False)
csv_logger = CSVLogger(time.strftime("%Y-%m-%d_%H%M-")+'training.log')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1, mode='auto')


# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential([
        Reshape(img_2Dshape, input_shape=(nb_dim,)),
        
        ZeroPadding2D(padding=(3, 3)),
        Convolution2D(16,7,7,init='he_normal'),
        Activation('relu'),
        BatchNormalization(),
        ZeroPadding2D(padding=(2, 2)),
        Convolution2D(32,5,5,init='he_normal'),
        Activation('relu'),
        BatchNormalization(),
        
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)), #Post-Size 14x14

        ZeroPadding2D(padding=(2, 2)),
        Convolution2D(64,5,5,init='he_normal'),
        Activation('relu'),
        BatchNormalization(),
        ZeroPadding2D(padding=(1, 1)),
        Convolution2D(128,3,3,init='he_normal'),
        Activation('relu'),
        BatchNormalization(),
        
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)), #Post-Size 7x7
        
        Convolution2D(4096,7,7,init='he_normal'),
        Activation('relu'),
        Convolution2D(nb_classes,1,1,init='he_normal'),
        Flatten(),
        Activation('softmax')
    ])
    
   
    # Compile model
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

clf_keras = KerasClassifier(build_fn=create_model, nb_epoch=256, batch_size=256, verbose=1,
                            validation_split=0.1,
                            callbacks=[history,checkpointer,csv_logger,early_stop])

# fix random seed for reproducibility
seed = 777
np.random.seed(seed)

In [6]:
featurelist = (''.join(['pixel',str(i)]) for i in range(784))
#featurelist generator

In [7]:
mapper = DataFrameMapper(
    [ ([pixel], StandardScaler()) for pixel in featurelist]
)
# Do we need StandardScaler() ? It's the same scale of color for all pixels

In [ ]:
pipe = Pipeline([
    ("featurize", mapper),
    ("mlp", clf_keras)
])

In [8]:
########################Helper functions ################
##### Cross Validation
def crossval():
    # split = ShuffleSplit(n_splits=10) #compat issue with sklearn pandas "object is not iterable"
    cv = cross_val_score(pipe, X_train, y_train, cv=10, n_jobs=1) #Note : don't use parallel crossval  and GPU computation
    print("Cross Validation Scores are: ", cv.round(3))
    print("Mean CrossVal score is: ", round(cv.mean(),3))
    print("Std Dev CrossVal score is: ", round(cv.std(),3))
    return cv
def output():
    predictions = pipe.predict(test)
    result = pd.DataFrame(predictions)
    result.index+=1
    result.index.names=['ImageId']
    result.columns=['Label']
    result.to_csv(time.strftime("%Y-%m-%d_%H%M-") + 'keras.csv', header=True)

In [9]:
X_train = train
y_train = train['label']

In [ ]:
temp_cv = crossval()

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
output()

If computation was interrupted and need to restart, we can't use a Pipeline right away (unless cPickled) because sklearn complains about the "StandardScaler" not being trained.

So we need to retrain the pipeline but freeze all the layers in the CNN

In [11]:
from keras.models import load_model
model=load_model('2017-01-26_0356-DCNN_weights.hdf5')

In [15]:
model.get_config()

[{'class_name': 'Reshape',
  'config': {'batch_input_shape': (None, 784),
   'input_dtype': 'float32',
   'name': 'reshape_1',
   'target_shape': (1, 28, 28),
   'trainable': True}},
 {'class_name': 'Convolution2D',
  'config': {'W_constraint': None,
   'W_regularizer': None,
   'activation': 'linear',
   'activity_regularizer': None,
   'b_constraint': None,
   'b_regularizer': None,
   'bias': True,
   'border_mode': 'valid',
   'dim_ordering': 'th',
   'init': 'glorot_uniform',
   'name': 'convolution2d_1',
   'nb_col': 7,
   'nb_filter': 16,
   'nb_row': 7,
   'subsample': (1, 1),
   'trainable': True}},
 {'class_name': 'Activation',
  'config': {'activation': 'relu', 'name': 'activation_1', 'trainable': True}},
 {'class_name': 'BatchNormalization',
  'config': {'axis': -1,
   'beta_regularizer': None,
   'epsilon': 0.001,
   'gamma_regularizer': None,
   'mode': 0,
   'momentum': 0.99,
   'name': 'batchnormalization_1',
   'trainable': True}},
 {'class_name': 'Convolution2D',
  'c

In [31]:
for layer in model.layers:
    layer.trainable = False
    
def call_model():
    return model

In [32]:
clf_keras_fromsave = KerasClassifier(build_fn=call_model, nb_epoch=1, batch_size=1024, verbose=1)

In [33]:
pipe = Pipeline([
    ("featurize", mapper),
    ("dcnn", clf_keras_fromsave)
])

In [34]:
pipe.fit(X_train, y_train)

/usr/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Epoch 1/1
42000/42000 [==============================] - 40s - loss: 0.0025 - acc: 0.9998         

Pipeline(steps=[('featurize', DataFrameMapper(default=False,
        features=[(['pixel0'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['pixel1'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['pixel2'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['pixel3'], S...   sparse=False)), ('dcnn', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fe42f17f048>)])

In [35]:
output()

/usr/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


28000/28000 [==============================] - 6s     